# Chapter 19 Dynamic Attributes and Properties

### __Dynamic Attributes 란?__  
동적 타입 언어인 파이썬의 강점은 속성 접근을 동적으로 할 수 있는 데에 있다. 동적 속성은 객체나 인스턴스를 생성한 이후, 런타임에 정의되는 속성을 말한다. 

In [1]:
class A:
    None

a = A()
a.d = 1

print(a.d)

1


인터프리터는 `obj.attr`과 같은 dot notation을 사용한 속성 접근을 위해 `__getattr__`, `__setattr__`과 같은 special method를 호출한다. 특히, 유저가 `__getattr__`을 따로 구현한 클래스의 경우, 존재하지 않는 속성에 대해 접근하려고 할 때 해당 값을 즉석에서 계산하기도 한다. 

JSON 형태의 데이터를 다룰 때 `feed['Schedule']['events'][40]['name']`과 같은 표현은 `feed.Schedule.events[40].name`과 같은 표현보다 쓰기 번거롭다. Python에서는 이러한 dot notation(attribute notation)을 사용가능하게 하는 클래스를 쉽게 구현해서 사용할 수 있다. 

__How?__

`__getattr__` special method 활용한다.  
- `__getattr__`은 인스턴스 -> 클래스 -> 부모 클래스 순서로 해당 attribute가 존재하는지 확인하여 어느 것에도 존재하지 않을 경우에 호출되는 함수이다. 

dictionary가 아닌 클래스가 dictionary의 method를 class의 attribute로 가질 수 있도록 구현해보자.

In [2]:
class NotDict:
    def __init__(self, mapping):
        self.__data = dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)


d = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
a = NotDict(d)

a.keys()

dict_keys(['key1', 'key2', 'key3'])

여기서 조금 더 발전 된 형태가 교재에 나와있는 예시와 유사하다. 위의 예시에서 dictionary의 method를 확인한 뒤, 존재하지 않을 때 dictionary의 key를 조사하도록 구현하여 클래스가 dictionary의 key도 속성으로 가질 수 있도록 할 수 있다.

In [3]:
class NotDict2:
    def __init__(self, mapping):
        self.__data = dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return self.__data[name]


d = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
a = NotDict2(d)

a.key2

'value2'

__주의__  
- `__getattr__`가 호출되는 과정이 인스턴스 -> 클래스 -> 부모 클래스 순서로 해당 attribute가 존재하는지 확인 후 호출된다고 했으니, 해당 자료형에 존재할 법한 이름(keyword 모듈 import해서 `keyword.iskeyword(key)`로 확인 가능)을 JSON data의 field 명으로 사용하면 오류가 날 것이다! (뒤에 `_`를 붙이는 것으로 대충 해결 가능...)  
- 또, 숫자로 시작하는 field도 조심! (파이썬의 변수명은 숫자로 시작 못하니까) 이렇게 파이썬의 변수명으로 사용가능 여부는 `str.isidentifier()`로 확인 가능

cf) 생성자로 객체생성을 호출받으면 먼저 `__new__` 를 호출하여 객체를 생성 할당하고, `__new__` 메소드의 반환값을 `__init__` 메소드의 첫번째 인자로 넘겨 호출하여 객체에서 사용할 초기값들을 초기화하게된다. 따라서 `__new__` 메소드를 수정해서, 들어오는 인자의 타입에 따라 다른 전처리를 할 수 있도록 구현할 수 있다. 

### __Property란?__  
attribute에 직접적으로 접근하는 것을 막기 위해 property를 decorator로 사용한다. 사용하면서 기대하는 효과는 다음 2가지다.  
1. 변수 변경 시 제한사항을 두고 싶은 경우  
2. getter, setter 함수를 만들지 않고 간단하게 접근하기 위해  

`@property` decorator를 사용하면 위의 2가지를 만족하면서, 기존의 인터페이스는 건드리지 않고 그대로 사용할 수 있다. (즉, 기존에 값을 설정하기 위해 `obj.attr = value`라는 식을 작성했다면 해당 데코레이터 사용 이후에도 똑같은 식을 사용하여 값을 설정할 수 있다는 것이다!)

In [29]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self): 
        return self.__weight
        
    @weight.setter
    def weight(self, value): 
        if value > 0:
            self.__weight = value 
        else:
            raise ValueError('value must be > 0')

In [5]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

위의 코드에서 weight에 제한사항을 두었지만, price도 음수가 되어서는 안 된다. 그래서 price에도 똑같은 조건을 걸어주어야하는데 그렇게 되면 같은 코드가 여러 번 쓰이게 된다. 이를 피하는 방법은 2가지다.  
1. property factory 사용  
2. descriptor class 사용(chapter 20에서 다룸)

__1. property factory 사용__  
보통 데코레이터로 사용되지만 property는 사실 클래스다. property 생성자의 완전한 모습은 `property(fget=None, fset=None, fdel=None, doc=None)`이다. 모든 인자들은 선택사항이지만, 해당 인자가 주어지지 않으면 결과로 나온 property 객체에서의 해당 연산도 제한된다.  

 - fdel은 attribute 삭제 함수로, decorator로 구현할 경우 `@property.deleter`을 사용하여 구현할 수 있고, fget함수를 통해 반환되는 것을 삭제하는 연산을 수행한다.  
 - doc은 property의 documentation을 나타내고, 이렇게 객체에 명시할 수도 있지만, getter함수에 docstring으로 명시할 수도 있다.  

property는 python 2.2, decorator는 python 2.4에 도입되어 사이 기간 동안은 property를 이 방법으로 사용했었다. 

In [6]:
class LineItem2:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    def get_weight(self): 
        return self.__weight
        
    def set_weight(self, value): 
        if value > 0:
            self.__weight = value 
        else:
            raise ValueError('value must be > 0')
    
    weight = property(get_weight, set_weight)

public class attribute에 property 객체를 넘겨줌으로서 사용 가능하다.

__신기한 점__

In [7]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'
    
obj = Class()
obj.__dict__

{}

In [8]:
obj.data

'the class data attr'

In [10]:
obj.data = 'bar'
obj.__dict__

{'data': 'bar'}

In [11]:
obj.data

'bar'

In [12]:
Class.data

'the class data attr'

보통 클래스 속성을 오버라이드하면 클래스 속성보다 인스턴스 속성이 더 우선한다. 따라서 위의 결과가 나온다. __하지만,__

In [13]:
Class.prop

In [14]:
obj.prop

'the prop value'

In [15]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [16]:
obj.__dict__['prop'] = 'foo'

In [17]:
obj.__dict__

{'data': 'bar', 'prop': 'foo'}

In [18]:
obj.prop

'the prop value'

In [19]:
Class.prop = 'baz'

In [20]:
obj.prop

'foo'

위의 예시에서 property는 인스턴스 속성에 의해 덮어써지지 않는 것을 확인할 수 있다. `obj.__dict__`에 직접적으로 속성을 추가하는 것은 가능하지만, 해당 property를 의미 없는 값으로 바꿔주고 나서야 인스턴스 속성이 드러난 것을 확인할 수 있다. 

In [21]:
obj.data

'bar'

In [22]:
Class.data

'the class data attr'

In [23]:
Class.data = property(lambda self: 'the "data" prop value')

In [24]:
obj.data

'the "data" prop value'

In [25]:
del Class.data

In [26]:
obj.data

'bar'

그리고 클래스 메소드를 property로 바꾸면 그 영향이 인스턴스에까지 미쳐서 인스턴스 속성을 덮어버리는 것을 확인할 수 있다. 

여기서 중요하게 확인할 것은 `obj.attr`은 인스턴스 `obj`부터 확인하는 것이 아니라, `obj.__class__`에서 해당 attr 이름을 가진 property가 있는지 먼저 확인을 하고 나서 인스턴스 속성을 확인한다는 것이다. 더 자세한 내용은 20장에서 다룬다.

이제 본론으로 돌아가 property factory를 사용하여 코드의 중복을 줄이는 방법을 알아보자.

In [27]:
def quantity(storage_name): 
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value): 
        if value > 0:
            instance.__dict__[storage_name] = value 
        else:
            raise ValueError('value must be > 0') 
    
    return property(qty_getter, qty_setter)

class LineItem3:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

`__getattr__` vs `__getattribute__`  
후자는 해당 attribute가 special attribute 혹은 special method 일 때를 제외하고 항상 호출된다. 전자는 후자가 이 attribute를 인스턴스나 클래스 내에서 찾는 것을 실패해 AttributeError를 반환할 때에만 호출된다. 주의할 점은 `__getattribute__` 메소드 내에 `self.attr`과 같이 해당 인스턴스 내의 속성에 또 접근하는 코드가 있을 경우, 재귀 호출을 무한으로 할 수 있기 때문에 이러한 상황에서는 `super().__getattribute__(obj, name)`을 사용해야한다. 